In [ ]:
#!/usr/bin/env bash
set -euo pipefail

# ------------------------------------------------------------------
# Script de interação com repositório GitHub via Git Bash
# Fluxo cronológico de uso para usuários finais no TCC:
# 1) Clone do repositório no servidor/local
# 2) Atualização de mudanças existentes (pull)
# 3) Download de arquivos específicos ou de pastas inteiras
# 4) Criação de branch para novas contribuições
# 5) Adição e commit de novos arquivos ou alterações
# 6) Envio (push) da branch ao GitHub para revisão (Pull Request manual)
# ------------------------------------------------------------------

# CONFIGURAÇÃO: ajuste para o repositório do seu TCC
REPO_URL="https://github.com/SeuUsuario/seu-repo.git"
LOCAL_DIR="seu-repo"

# Exibe instruções de uso
usage() {
  cat <<EOF
Uso: $0 <comando> [args]
Comandos em ordem cronológica:
  clone                     Clonar o repositório (passo 1)
  pull                      Atualizar com as últimas mudanças (passo 2)
  download-file <caminho>   Baixar arquivo específico (passo 3)
  download-folder <pasta>   Baixar toda uma pasta usando sparse-checkout (passo 3)
  branch <nome-branch>      Criar e trocar para nova branch (passo 4)
  commit "<mensagem>"       Adicionar todas as mudanças e criar commit (passo 5)
  push                      Enviar branch atual ao GitHub (passo 6)
EOF
  exit 1
}

case "${1:-}" in
  clone)
    # Passo 1: Clonar o repositório GitHub no diretório local
    echo "Passo 1: Clonando repositório em ./${LOCAL_DIR}..."
    git clone "$REPO_URL" "$LOCAL_DIR"
    echo "Repositório clonado com sucesso."
    ;;

  pull)
    # Passo 2: Atualizar o repositório local com mudanças do remoto
    cd "$LOCAL_DIR"
    echo "Passo 2: Obtendo e integrando mudanças do GitHub..."
    git fetch origin
    git pull --ff-only
    echo "Repositório atualizado."
    ;;

  download-file)
    # Passo 3a: Baixar arquivo específico do branch main
    FILE_PATH="${2:-}"
    if [[ -z "$FILE_PATH" ]]; then
      echo "Erro: especifique o caminho do arquivo dentro do repositório."
      exit 1
    fi
    RAW_URL="https://raw.githubusercontent.com/SeuUsuario/seu-repo/main/${FILE_PATH}"
    echo "Passo 3: Baixando arquivo '$FILE_PATH'..."
    curl -L "$RAW_URL" -o "$(basename "$FILE_PATH")"
    echo "Arquivo baixado: $(basename "$FILE_PATH")."
    ;;

  download-folder)
    # Passo 3b: Baixar pasta inteira com sparse-checkout
    FOLDER="${2:-}"
    if [[ -z "$FOLDER" ]]; then
      echo "Erro: especifique o nome da pasta a baixar."
      exit 1
    fi
    echo "Passo 3: Clonando repositório de forma leve para pasta '$FOLDER'..."
    # Clona sem checkout completo
    git clone --no-checkout "$REPO_URL" "${LOCAL_DIR}-sparse"
    cd "${LOCAL_DIR}-sparse"
    git sparse-checkout init --cone
    git sparse-checkout set "$FOLDER"
    git checkout main
    cd ..
    mv "${LOCAL_DIR}-sparse/$FOLDER" "./$FOLDER"
    rm -rf "${LOCAL_DIR}-sparse"
    echo "Pasta '$FOLDER' baixada com sucesso."
    ;;

  branch)
    # Passo 4: Criar e trocar para uma nova branch de trabalho
    BRANCH="${2:-}"
    if [[ -z "$BRANCH" ]]; then
      echo "Erro: especifique o nome da nova branch."
      exit 1
    fi
    cd "$LOCAL_DIR"
    echo "Passo 4: Criando e mudando para branch '$BRANCH'..."
    git checkout -b "$BRANCH"
    echo "Branch '$BRANCH' criada e selecionada."
    ;;

  commit)
    # Passo 5: Adicionar mudanças e criar um commit
    MESSAGE="${2:-}"
    if [[ -z "$MESSAGE" ]]; then
      echo "Erro: forneça a mensagem do commit entre aspas."
      exit 1
    fi
    cd "$LOCAL_DIR"
    echo "Passo 5: Adicionando alterações e criando commit..."
    git add .
    git commit -m "$MESSAGE"
    echo "Commit criado com a mensagem: $MESSAGE"
    ;;

  push)
    # Passo 6: Enviar a branch atual para o GitHub
    cd "$LOCAL_DIR"
    CURRENT_BRANCH=$(git rev-parse --abbrev-ref HEAD)
    echo "Passo 6: Enviando branch '$CURRENT_BRANCH' para o GitHub..."
    git push -u origin "$CURRENT_BRANCH"
    echo "Branch '$CURRENT_BRANCH' enviada. Agora abra um Pull Request via GitHub Web."
    ;;

  *)
    usage
    ;;
esac

echo "Fluxo concluído. Próximos passos: no GitHub, abra um Pull Request para revisão."
